# Tutorial with Re-Rankers

This notebook shows how post-hoc experiments of the IR Experiment Platform can be conducted.

To start the notebook, please clone the archived shared task repository:

```
git@github.com:tira-io/ir-experiment-platform-benchmarks.git
```

Inside the cloned repository, you can start the Jupyter notebook which automatically installs a minimal virtual environment using:
```
make jupyterlab
```

The notebook covers how to run re-rankers submitted to TIRA in reproducibility/replicability experiments on the same or new data.

For each of the softwares submitted to TIRA, the `tira` integration to PyTerrier loads the Docker Image submitted to TIRA to execute it in PyTerrier pipelines (i.e., a first execution could take sligthly longer).


## Import Dependencies

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)

from tira.local_client import Client
tira = Client()

import pyterrier as pt
if not pt.started():
    pt.init()


### Initialize A Re-Ranker

We create a pyterrier re-ranker called `advanced_pipeline` that re-ranks BM25 with an re-ranker submitted to a shared task in TIRA.
The reranker is identified by the name `ir-benchmarks/tira-ir-starter/SBERT multi-qa-MiniLM-L6-dot-v1 (tira-ir-starter-beir)`, i.e., a software `SBERT multi-qa-MiniLM-L6-dot-v1 (tira-ir-starter-beir)` submitted to `ir-benchmarks` by the team `tira-ir-starter` (that hosts baselines).
This software consists a single stage that re-ranks with a dense retrieval approach implemented in BEIR.

With this API, any re-rank approach submitted in TIRA can be executed and re-executed, e.g., on new data.

We can run the re-ranker on any dataset integrated in `ir_dataset` or any dataframe.
Here, we use a small artificial reranking dataset `data_to_rerank` to show the overall functionality with a fast example.

In [3]:
data_to_rerank = pd.DataFrame([
        ["d1", "this is the first document of many documents", "1", "first document"],
        ["d2", "this is another document", "1", "first document"],
        ["d3", "the topic of this document is unknown", "1", "first document"]
    ], columns=["docno", "body", "qid", "query"])

data_to_rerank

,docno,body,qid,query
0,d1,this is the first document of many documents,1,first document
1,d2,this is another document,1,first document
2,d3,the topic of this document is unknown,1,first document


First, we re-rank this via BM25.

In [4]:
bm25 = pt.text.scorer(wmmodel='bm25')

bm25(data_to_rerank)

,docno,body,qid,rank,score,query
0,d1,this is the first document of many documents,1,0,5.560003e-01,first document
1,d2,this is another document,1,2,-3.085859e-10,first document
2,d3,the topic of this document is unknown,1,1,5.681316e-02,first document


Next, we use the re-ranker `ir-benchmarks/tira-ir-starter/SBERT multi-qa-MiniLM-L6-dot-v1 (tira-ir-starter-beir)` as second stage re-ranker, after BM25.

This shows that re-rankers in TIRA are interoperable with other re-rankers.


In [5]:
advanced_pipeline = bm25 >> tira.pt.reranker('ir-benchmarks/tira-ir-starter/SBERT multi-qa-MiniLM-L6-dot-v1 (tira-ir-starter-beir)')

advanced_pipeline(data_to_rerank)

,docno,body,qid,query,q0,rank,score,system
0,d1,this is the first document of many documents,1,first document,0,1,46.084885,multi-qa-MiniLM-L6-dot-v1-dot
1,d2,this is another document,1,first document,0,2,40.802025,multi-qa-MiniLM-L6-dot-v1-dot
2,d3,the topic of this document is unknown,1,first document,0,3,37.294750,multi-qa-MiniLM-L6-dot-v1-dot
